## Calculate benchmarking scores

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad

import INSPIRE

import warnings
warnings.filterwarnings("ignore")

### Load results

In [2]:
res_dir = "Results/INSPIRE_DLPFC"
adata = sc.read_h5ad(res_dir + "/adata_inspire.h5ad")
basis_df = pd.read_csv(res_dir + "/basis_df_inspire.csv", index_col=0)
basis = np.array(basis_df.values)

In [3]:
adata

AnnData object with n_obs × n_vars = 14243 × 13988
    obs: 'in_tissue', 'array_row', 'array_col', 'barcode', 'slice_id', 'layer', 'n_genes', 'slice', 'Proportion of spatial factor 1', 'Proportion of spatial factor 2', 'Proportion of spatial factor 3', 'Proportion of spatial factor 4', 'Proportion of spatial factor 5', 'Proportion of spatial factor 6', 'Proportion of spatial factor 7', 'Proportion of spatial factor 8', 'Proportion of spatial factor 9', 'Proportion of spatial factor 10', 'Proportion of spatial factor 11', 'Proportion of spatial factor 12', 'Proportion of spatial factor 13', 'Proportion of spatial factor 14', 'Proportion of spatial factor 15', 'Proportion of spatial factor 16', 'Proportion of spatial factor 17', 'Proportion of spatial factor 18', 'Proportion of spatial factor 19', 'Proportion of spatial factor 20', 'GM'
    uns: 'layer_colors', 'slice_colors'
    obsm: 'X_umap', 'latent', 'spatial'

### Calculate ASW, ARI, and NMI

In [4]:
n_slices = len(set(adata.obs.slice.values))

In [5]:
# ASW
for i in range(n_slices):
    adata_tmp = adata[adata.obs.slice.values.astype(str) == str(i), :]
    ASW = INSPIRE.utils.calculate_ASW(adata_tmp.obsm["latent"], adata_tmp.obs["layer"].astype(str))
    print("slice", str(i), "ASW =", ASW)

slice 0 ASW = 0.21675144
slice 1 ASW = 0.23563725
slice 2 ASW = 0.20992284
slice 3 ASW = 0.21229379


In [6]:
# ARI
for i in range(n_slices):
    adata_tmp = adata[adata.obs.slice.values.astype(str) == str(i), :]
    ARI = INSPIRE.utils.calculate_ARI(adata_tmp.obs["layer"].astype(str), adata_tmp.obs["GM"])
    print("slice", str(i), "ARI =", ARI)

slice 0 ARI = 0.5760796384927773
slice 1 ARI = 0.6430556022130648
slice 2 ARI = 0.6137699726059762
slice 3 ARI = 0.6139559439969932


In [7]:
# NMI
for i in range(n_slices):
    adata_tmp = adata[adata.obs.slice.values.astype(str) == str(i), :]
    NMI = INSPIRE.utils.calculate_NMI(adata_tmp.obs["layer"].astype(str), adata_tmp.obs["GM"])
    print("slice", str(i), "NMI =", NMI)

slice 0 NMI = 0.7143137356477687
slice 1 NMI = 0.7319254540318172
slice 2 NMI = 0.72224978876709
slice 3 NMI = 0.712963408054093


### Calculate factor diversity and factor coherence

In [8]:
# factor diversity
factor_diversity = INSPIRE.utils.calculate_factor_diversity(basis, n_top_genes=10)
print("factor_diversity =", factor_diversity)

factor_diversity = 0.525


In [9]:
# get count matrix
data_dir = "data/DLPFC/spatialLIBD"
adata_st_list = []

slice_idx = 151673
adata_st = sc.read_visium(path=data_dir+"/%d" % slice_idx,
                          count_file="%d_filtered_feature_bc_matrix.h5" % slice_idx)
adata_st.obs.index = adata_st.obs.index + "-0"
adata_st.var_names_make_unique()
adata_st_list.append(adata_st)

slice_idx = 151674
adata_st = sc.read_visium(path=data_dir+"/%d" % slice_idx,
                          count_file="%d_filtered_feature_bc_matrix.h5" % slice_idx)
adata_st.obs.index = adata_st.obs.index + "-1"
adata_st.var_names_make_unique()
adata_st_list.append(adata_st)

slice_idx = 151675
adata_st = sc.read_visium(path=data_dir+"/%d" % slice_idx,
                          count_file="%d_filtered_feature_bc_matrix.h5" % slice_idx)
adata_st.obs.index = adata_st.obs.index + "-2"
adata_st.var_names_make_unique()
adata_st_list.append(adata_st)

slice_idx = 151676
adata_st = sc.read_visium(path=data_dir+"/%d" % slice_idx,
                          count_file="%d_filtered_feature_bc_matrix.h5" % slice_idx)
adata_st.obs.index = adata_st.obs.index + "-3"
adata_st.var_names_make_unique()
adata_st_list.append(adata_st)

adata_raw = ad.concat(adata_st_list)
adata_raw = adata_raw[adata.obs.index, :]
adata_raw = adata_raw[:, basis_df.columns]

In [10]:
# factor coherence
factor_coherence = INSPIRE.utils.calculate_factor_coherence(basis, n_top_genes=10, gene_counts=np.array(adata_raw.X.todense()))
print("factor_coherence =", factor_coherence)

factor_coherence = 0.11904762806481868
